In [18]:
import sys
import ftplib
import os
import time
import re
from datetime import datetime
import logging
from logging.handlers import RotatingFileHandler

import smtplib
from email.message import EmailMessage

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive





with open('password.txt') as file:  
    EmailPassword = file.read()
    
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')
LOG_FILENAME = datetime.now().strftime('Ranjeet/PSALE/Log/PSALE_US_FTP_Download_logfile_%H_%M_%S_%d_%m_%Y.log')
file_handler = RotatingFileHandler(LOG_FILENAME,mode='w')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
file_handler.flush()
logger.addHandler(file_handler)


server = "edi.lightningsource.com"
user = "blurusord"
password = "blurus0109"
source = "outgoing"
destination = "/Users/rkumar/Ranjeet/PSALE"

FromEmail='ranjeetkumarpython@gmail.com'
ToEmail='ranjeetkumarpython@gmail.com'



def email_notification(FromEmail,ToEmail,EmailPassword,Subject,Message):
    msg = EmailMessage()
    msg['Subject'] = Subject
    msg['From'] = FromEmail
    msg['To'] = ToEmail
    msg.set_content(Message)
    s = smtplib.SMTP('smtp.gmail.com',587)
    s.ehlo()
    s.starttls()
    s.login(FromEmail,EmailPassword)
    try:
        s.send_message(msg)
        logger.debug('Email Sent')
    except:
        logger.exception('There was problem in Email Push')
        
        
        

def ftp_login(server,user,password,source):
    try:
        ftp = ftplib.FTP(server)
        ftp.login(user, password)
        logger.debug('Successfully logged in to {}'.format(server))
    except Exception as e:
        logger.exception('FTP Server login error:')
        email_notification(FromEmail,ToEmail,EmailPassword,'FTP Failed','FTP Login Failed')
        
    ftp.cwd(source)
    files = []
    files = ftp.nlst()
    fnames=list(filter(lambda x:"PSALES." in x and '.xls' in x ,files))
    latest_time = None
    latest_name = None
    for name in fnames:
        time = ftp.voidcmd("MDTM " + name)
        if (latest_time is None) or (time > latest_time):
            latest_name = name
            latest_time = time
    logger.debug('{} Most recent file is ready to download.'.format(latest_name))
    #Download file
    local_filename = os.path.join(destination, latest_name)
    lf = open(local_filename, "wb")
    try:
        ftp.retrbinary("RETR " + latest_name, lf.write, 8*1024)
        lf.close()
        ftp.quit()
        filesize = os.path.getsize(local_filename)
        logger.debug('{} File Successfully downloaded. File size: {} Bytes'.format(latest_name,filesize))
    except Exception as e:
        logger.exception('Could not download {} file'.format(latest_name))
        Message=f'Could not download {latest_name}'
        email_notification(FromEmail,ToEmail,EmailPassword,'File Download Failed','FTP Login Failed')
    return latest_name
    

    
def googledriveupload(destination,latest_name):
    try:
        gauth = GoogleAuth()
        gauth.LoadCredentialsFile('clientsecrets.txt')
        if gauth.credentials is None:
            gauth.LocalWebserverAuth()
        elif gauth.access_token_expired:
            gauth.Refresh()
        else:
            gauth.Authorize()
        gauth.SaveCredentialsFile('clientsecrets.txt')
        drive = GoogleDrive(gauth)
        logger.debug('Google drive Authentication successful.')
    except Exception as e:
        logger.exception('Google drive Authentication FAILED.')
        email_notification(FromEmail,ToEmail,EmailPassword,'Google drive Authentication FAILED.','Google drive Authentication FAILED.')

    #destination="/Users/rkumar/Ranjeet/PSALE"
    #latest_name='PSALES.202002251805484905058.xls'
    local_filename = os.path.join(destination, latest_name)
    YYYYMM=datetime.today().strftime('%Y%m')
    upload_file="PSALES_"+YYYYMM
    try:
        file_drive = drive.CreateFile({'title': upload_file })
        file_drive.SetContentFile(local_filename) 
        file_drive.Upload()
        logger.debug('{} file successfully uploaded in google drive as {}'.format(latest_name,upload_file))
    except Exception as e:
        logger.exception('{} file could not uploaded in google drive.'.format(latest_name))
        email_notification(FromEmail,ToEmail,EmailPassword,'File upload falied','File upload failed in google drive.')
    return upload_file

    
#if __name__ == "__main__":    
latest_name=ftp_login(server,user,password,source)

googledriveupload(destination,latest_name)


    

'PSALES_202003'